In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

## Imports

In [ ]:
import os
import random
import warnings
from contextlib import redirect_stdout
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

plt.rc('font', size=16)

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Setting Random seed for reproducibility

In [ ]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

## Utils Methods

In [ ]:
def build_sequences(df, target_labels, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset_local = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df) % window

    if padding_len != 0:
        # Compute padding length
        padding_len = window - len(df) % window
        padding = np.zeros((padding_len, temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding, df))
        padding = np.zeros((padding_len, temp_label.shape[1]), dtype='float32')
        temp_label = np.concatenate((padding, temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0, len(temp_df) - window - telescope, stride):
        dataset_local.append(temp_df[idx:idx + window])
        labels.append(temp_label[idx + window:idx + window + telescope])

    dataset_local = np.array(dataset_local)
    labels = np.array(labels)
    return dataset_local, labels

In [ ]:
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if idx is None:
        idx = np.random.randint(0, len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17, 17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0, :, i])), X[idx, :, i])
        axs[i].scatter(np.arange(len(X[0, :, i]), len(X[0, :, i]) + telescope), y[idx, :, i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0, 1)
    plt.savefig("multivariate", bbox_inches='tight')
    plt.show()


def inspect_multivariate_prediction(X_train, X, y, pred, columns, telescope, idx=None):
    if idx is None:
        idx = np.random.randint(0, len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17, 17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0, :, i])), X[idx, :, i])
        axs[i].plot(np.arange(len(X[0, :, i]), len(X_train[0, :, i]) + telescope), y[idx, :, i], color='orange')
        axs[i].plot(np.arange(len(X[0, :, i]), len(X_train[0, :, i]) + telescope), pred[idx, :, i], color='green')
        axs[i].set_title(col)
        axs[i].set_ylim(0, 1)
    plt.savefig("multivariate_prediction", bbox_inches='tight')
    plt.show()

## Metadata

In [ ]:
model_name = 'ATTENTION_LSTM_512_800_5_T_144'
%cd /gdrive/My Drive/Colab Notebooks
!mkdir ATTENTION_LSTM_512_800_5_T_144

# metadata
window = 800
stride = 5
telescope = 144
batch_size = 128
epochs = 200
reg_telescope = 864

## Load the Data

In [ ]:
# Load the dataset
%cd /gdrive/My Drive/Colab Notebooks/data
dataset = pd.read_csv("dataset.csv")
target_labels = dataset.columns

%cd /gdrive/My Drive/Colab Notebooks/ATTENTION_LSTM_512_800_5_T_144
# Print dataset info
dataset.info()

## Inspect the dataframe

In [ ]:
figs, axs = plt.subplots(len(dataset.columns), 1, sharex=True, figsize=(17, 17))
for i, col in enumerate(dataset.columns):
    axs[i].plot(dataset[col])
    axs[i].set_title(col)
plt.savefig("inspect_dataframe", bbox_inches='tight')
plt.show()

## Split the Data

In [ ]:
# ASSICURARSI CHE LA TEST SIZE SIA MAGGIORE DELLA WINDOW! IN CASO SERVA PIU SPAZIO INCREMENTARE LA TEST_SIZE
test_size = int(dataset['Sponginess'].size * 0.1)

print('Test raw size: ' + str(test_size))
X_train_raw = dataset.iloc[:-test_size]
X_test_raw = dataset.iloc[-test_size:]
print("Train raw shape: " + str(X_train_raw.shape) + "  Test raw shape: " + str(X_test_raw.shape))

# Normalize both features and labels
X_min = X_train_raw.min()
X_max = X_train_raw.max()

X_train_raw = (X_train_raw - X_min) / (X_max - X_min)
X_test_raw = (X_test_raw - X_min) / (X_max - X_min)

plt.figure(figsize=(17, 5))
plt.plot(X_train_raw.Sponginess, label='Train (Sponginess)')
plt.plot(X_test_raw.Sponginess, label='Test (Sponginess)')
plt.title('Train-Test Split')
plt.legend()
plt.savefig("Train-Test_Split", bbox_inches='tight')
plt.show()

future = dataset[-window:]
future = (future - X_min) / (X_max - X_min)
future = np.expand_dims(future, axis=0)
print("Future shape: " + str(future.shape))

In [ ]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_test, y_test = build_sequences(X_test_raw, target_labels, window, stride, reg_telescope)
print("X Train shape: " + str(X_train.shape) + " Y Train shape: " + str(y_train.shape))
print("X Test shape: " + str(X_test.shape) + " Y Test shape: " + str(y_test.shape))

inspect_multivariate(X_train, y_train, target_labels, telescope)

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

## Build the model

### Model metadata

In [ ]:
dropout_rate = .3
units = 512 

### Model building

In [ ]:
from keras import backend as K
class Attention(tfkl.Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = tfk.initializers.get('glorot_uniform')

        self.W_regularizer = tfk.regularizers.get(W_regularizer)
        self.b_regularizer = tfk.regularizers.get(b_regularizer)

        self.W_constraint = tfk.constraints.get(W_constraint)
        self.b_constraint = tfk.constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [24]:
# Build the neural network layer by layer
input_layer = tfkl.Input(shape=input_shape, name='Input')

lstm = tfkl.Bidirectional(tfkl.LSTM(units, return_sequences=True),name='BILSTM01')(input_layer)
lstm = tfkl.Dropout(dropout_rate, name='DROP01')(lstm)

lstm = Attention(input_shape[0])(lstm)
lstm = tfkl.RepeatVector(window, name='REP')(lstm)

lstm = tfkl.TimeDistributed(tfkl.Dense(units, activation=tfk.layers.LeakyReLU(alpha=0.01)), name='TDDENSE01')(lstm)
lstm = tfkl.Dropout(dropout_rate, name='DROP02')(lstm)

lstm = tfkl.Bidirectional(tfkl.LSTM(units, return_sequences=True),name='BILSTM02')(lstm)
lstm = tfkl.Dropout(dropout_rate, name='DROP03')(lstm)

lstm = Attention(input_shape[0])(lstm)

# Output: [None, telescope, num_channels]
dense = tfkl.Dense(output_shape[-1] * output_shape[-2], activation=tfk.layers.LeakyReLU(alpha=0.01), name='DENSE04')(lstm)
output_layer = tfkl.Reshape((output_shape[-2], output_shape[-1]), name='RESHAPE')(dense)

# Connect input and output through the Model class
model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

# Compile the model
model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=tf.keras.metrics.RootMeanSquaredError(name="rmse"))

### Model summary

In [ ]:
model.summary()

## Start Training

In [ ]:
print("Starting training...")
# Train the model
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=.1,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1,
                                             mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
    ]
).history

# Save model
model.save(model_name)
print("Model saved!")
model = tfk.models.load_model(model_name)

### Plot the training history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17, 4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.savefig("val_loss", bbox_inches='tight')
plt.show()

plt.figure(figsize=(17, 4))
plt.plot(history['rmse'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_rmse'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Root Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.savefig("rmse", bbox_inches='tight')
plt.show()

plt.figure(figsize=(18, 3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.title("Learning Rate")
plt.savefig("learning_rate", bbox_inches='tight')
plt.show()

## Predict function for autoregressive models (Same for testing and predict the future)

In [ ]:
def predict(test_data):
    predictions = np.array([])
    X_temp = test_data
    for reg in range(0, reg_telescope, telescope):
        pred_temp = model.predict(X_temp)
        if len(predictions) == 0:
            predictions = pred_temp
        else:
            predictions = np.concatenate((predictions, pred_temp), axis=1)
        X_temp = np.concatenate((X_temp[:, telescope:, :], pred_temp), axis=1)
    return predictions

## Start Testing

In [ ]:
print("Testing...")
predictions = predict(X_test)

In [ ]:
denorm_predictions = (tf.convert_to_tensor(predictions, dtype=tf.float32) * (X_max - X_min)) + X_min
denorm_predictions = denorm_predictions.numpy()
denorm_y_test = (tf.convert_to_tensor(y_test, dtype=tf.float32) * (X_max - X_min)) + X_min
denorm_y_test = denorm_y_test.numpy()

mse = tfk.metrics.mse(denorm_y_test.flatten(),denorm_predictions.flatten()).numpy()
rmse = np.sqrt(mse)
print("MSE: "+ str(mse) +" RMSE: "+str(rmse))
inspect_multivariate_prediction(X_train, X_test, y_test, predictions, target_labels, reg_telescope)

In [ ]:
with open('modelsummary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()
        print("\nWINDOW: "+str(window)+" STRIDE: "+str(stride)+" BATCH SIZE: "+str(batch_size))
        print("\nMSE: "+ str(mse) +" RMSE: "+str(rmse))

## Predict the Future

### Maes

In [ ]:
maes = []
for i in range(predictions.shape[1]):
    ft_maes = []
    for j in range(predictions.shape[2]):
        ft_maes.append(np.mean(np.abs(y_test[:, i, j] - predictions[:, i, j]), axis=0))
    ft_maes = np.array(ft_maes)
    maes.append(ft_maes)
maes = np.array(maes)

### Predict

In [ ]:
print("Predict the future...")
future_predictions = predict(future)
print("Future Prediction shape: " + str(future_predictions.shape))

### Plot the future predictions

In [ ]:
figs, axs = plt.subplots(len(target_labels), 1, sharex=True, figsize=(17, 17))
for i, col in enumerate(target_labels):
    axs[i].plot(np.arange(len(future[0, :, i])), future[0, :, i])
    axs[i].plot(np.arange(len(future[0, :, i]), len(future[0, :, i]) + reg_telescope), future_predictions[0, :, i],
                color='orange')
    axs[i].fill_between(
        np.arange(len(future[0, :, i]), len(future[0, :, i]) + reg_telescope),
        future_predictions[0, :, i] + maes[:, i],
        future_predictions[0, :, i] - maes[:, i],
        color='orange', alpha=.3)
    axs[i].set_title(col)
plt.savefig("predict", bbox_inches='tight')
plt.show()